In [1]:
import pandas as pd
import pymysql
import re,json,os

In [2]:
base=os.environ['BUG_FREE_EUREKA_BASE'] 
pathTofile=os.path.join(base,'data','exercise06','E-TABM-713_A-MEXP-1551-analytics.tsv')


with open(os.path.join(base,'secrets.json')) as f:
    secrets = json.load(f)
    
conn = pymysql.Connect(**secrets['test_db']) # to use sql
cursor = conn.cursor() 

cursor.execute(' create database if not exists biodb')

df= pd.read_csv(pathTofile,sep='\t')



C:\Users\randa\Anaconda3\lib\site-packages\pymysql\cursors.py:166: Warning: (1007, "Can't create database 'biodb'; database exists")
  result = self._query(query)


In [3]:
# load data to dataframe:
df.head()

,Gene ID,Gene Name,Design Element,g1_g2.p-value,g1_g2.t-statistic,g1_g2.log2foldchange
0,MIMAT0000062,MIMAT0000062,A_25_P00010086,0.702506,0.625362,0.078492
1,MIMAT0000062,MIMAT0000062,A_25_P00011584,0.975514,0.065110,0.007410
2,MIMAT0000063,MIMAT0000063,A_25_P00010070,0.139014,2.431636,0.556112
3,MIMAT0000063,MIMAT0000063,A_25_P00010071,0.049728,3.254492,0.811621
4,MIMAT0000064,MIMAT0000064,A_25_P00010072,0.098283,2.697813,0.757066


In [4]:
#rename columns

df.columns=[re.sub('[ .-]','',x) for x in df.columns]
df.to_sql('dea',conn,flavor='mysql')

C:\Users\randa\Anaconda3\lib\site-packages\pandas\core\generic.py:1165: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  chunksize=chunksize, dtype=dtype)


In [5]:
# check if table is created 
cursor.execute("show tables")
cursor.fetchall()

(('columns_priv',),
 ('db',),
 ('dea',),
 ('engine_cost',),
 ('event',),
 ('func',),
 ('general_log',),
 ('gtid_executed',),
 ('help_category',),
 ('help_keyword',),
 ('help_relation',),
 ('help_topic',),
 ('innodb_index_stats',),
 ('innodb_table_stats',),
 ('ndb_binlog_index',),
 ('plugin',),
 ('proc',),
 ('procs_priv',),
 ('proxies_priv',),
 ('server_cost',),
 ('servers',),
 ('slave_master_info',),
 ('slave_relay_log_info',),
 ('slave_worker_info',),
 ('slow_log',),
 ('tables_priv',),
 ('time_zone',),
 ('time_zone_leap_second',),
 ('time_zone_name',),
 ('time_zone_transition',),
 ('time_zone_transition_type',),
 ('user',))

In [8]:
# write  a query to fetch the first 10 lines from dea
cursor.execute("""
Select *
from dea
limit 10""")

cursor.fetchall()

((0,
  'MIMAT0000062',
  'MIMAT0000062',
  'A_25_P00010086',
  0.70250559386657,
  0.625362183554288,
  0.0784915951074403),
 (1,
  'MIMAT0000062',
  'MIMAT0000062',
  'A_25_P00011584',
  0.975513746316429,
  0.0651101242194695,
  0.00741007045087905),
 (2,
  'MIMAT0000063',
  'MIMAT0000063',
  'A_25_P00010070',
  0.139013519150447,
  2.43163612513429,
  0.556111895299825),
 (3,
  'MIMAT0000063',
  'MIMAT0000063',
  'A_25_P00010071',
  0.0497282355402996,
  3.25449181729957,
  0.811621134771325),
 (4,
  'MIMAT0000064',
  'MIMAT0000064',
  'A_25_P00010072',
  0.098282650062664,
  2.69781280670979,
  0.757065824690136),
 (5,
  'MIMAT0000064',
  'MIMAT0000064',
  'A_25_P00010073',
  0.324778784391911,
  1.6485897929498,
  0.440504336234509),
 (6,
  'MIMAT0000065',
  'MIMAT0000065',
  'A_25_P00010281',
  0.103149521552763,
  2.66619309645875,
  0.382716406596201),
 (7,
  'MIMAT0000065',
  'MIMAT0000065',
  'A_25_P00010282',
  0.217890974906888,
  2.04707816939775,
  0.287800723552511),
 (8

In [24]:
# count the number of rows in the table
cursor.execute("""
Select count(*) as no_of_rows
from dea
""")

cursor.fetchall()

((1230,),)

In [13]:
# calculate the average p_value for the entire data
# first need to check the p_value name :
cursor.execute("desc dea")

cursor.fetchall()

(('index', 'bigint(20)', 'YES', 'MUL', None, ''),
 ('GeneID', 'varchar(63)', 'YES', '', None, ''),
 ('GeneName', 'varchar(63)', 'YES', '', None, ''),
 ('DesignElement', 'varchar(63)', 'YES', '', None, ''),
 ('g1_g2pvalue', 'double', 'YES', '', None, ''),
 ('g1_g2tstatistic', 'double', 'YES', '', None, ''),
 ('g1_g2log2foldchange', 'double', 'YES', '', None, ''))

In [14]:
# calculate the average p_value for the entire data
cursor.execute("""
Select avg(g1_g2pvalue)
from dea
 
""")

cursor.fetchall()

((0.4529776486719517,),)

In [25]:
# calculate the avg p_value for each gene
cursor.execute("""
Select avg(g1_g2pvalue),GeneID
from dea
group by GeneID
""")

cursor.fetchall()


((0.8390096700914995, 'MIMAT0000062'),
 (0.09437087734537329, 'MIMAT0000063'),
 (0.2115307172272875, 'MIMAT0000064'),
 (0.1605202482298255, 'MIMAT0000065'),
 (0.421072681151927, 'MIMAT0000066'),
 (0.3668396148775826, 'MIMAT0000067'),
 (0.172509684242204, 'MIMAT0000068'),
 (0.040891551036128196, 'MIMAT0000069'),
 (0.4276906754653405, 'MIMAT0000070'),
 (0.891390789294826, 'MIMAT0000071'),
 (0.32429179301055, 'MIMAT0000072'),
 (0.5245145043568785, 'MIMAT0000073'),
 (0.290672380835091, 'MIMAT0000074'),
 (0.464782884457078, 'MIMAT0000075'),
 (0.4530537439696015, 'MIMAT0000076'),
 (0.641912313397985, 'MIMAT0000077'),
 (0.0212501986719539, 'MIMAT0000078'),
 (0.718276941344944, 'MIMAT0000079'),
 (0.603024955133693, 'MIMAT0000080'),
 (0.2336861207054201, 'MIMAT0000081'),
 (0.0031096021726969205, 'MIMAT0000082'),
 (0.05413238635436655, 'MIMAT0000083'),
 (0.8315172458771201, 'MIMAT0000084'),
 (0.32132354018986153, 'MIMAT0000085'),
 (0.1032058916642123, 'MIMAT0000086'),
 (0.8714517409059265, 'MIMA

In [22]:
# find the gene with lowest fold change, lowest fold change

cursor.execute("""
Select GeneName
from dea
where g1_g2log2foldchange = 
(Select min(g1_g2log2foldchange)
from dea)
""")

cursor.fetchall()

(('MIMAT0000451',),)

In [23]:
# find the gene with highest fold change

cursor.execute("""
Select GeneName
from dea
where g1_g2log2foldchange = 
(Select max(g1_g2log2foldchange)
from dea)
""")

cursor.fetchall()

(('MIMAT0002177',),)